## Series de tiempo

In [3]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from tensorflow import keras
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import *

sns.set_style('whitegrid')

In [4]:
driver = 'postgres'
username = 'ds4a'
password = 'ds4A-7eaM79'
host = 'ds4a.cm1dcdf7pnnv.us-east-2.rds.amazonaws.com'
port = 5432
database = 'team_79'

connection_string = f'{driver}://{username}:{password}@{host}:{port}/{database}'
engine = create_engine(connection_string, encoding='utf8')

In [5]:
homicidios ='''
SELECT * FROM violencia_intrafamiliar
'''

data = pd.read_sql(homicidios, engine)

In [29]:
bogota = data[data.municipio.str.contains('BOGO').fillna(False)].copy()
# bogota = data
bogota.fecha = pd.to_datetime(bogota.fecha)
bogota_count = bogota.groupby('fecha').size().rename('num_cases')
resampled_date = bogota_count.resample('D').first().fillna(0)

In [50]:
X_train = resampled_date.iloc[0:int(resampled_date.shape[0]*0.8)]
X_test = resampled_date.iloc[int(resampled_date.shape[0]*0.8):]
arima_model = ARIMA(X_train, order=(2,1,1))
res = arima_model.fit()

In [51]:
preds = res.forecast(steps=X_test.shape[0])

In [52]:
mean_absolute_error(preds, X_test)

9.249336839070946